In [1]:
import socket
import threading
import numpy as np

In [2]:
def relu(x):
    return np.maximum(0, x)
def derrelu(x):
    return np.where(x > 0, 1, 0)
def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
def dertanh(x):
    return 1-np.power(np.tanh(x),2)
def sig(x):
    x = np.clip(x, -500, 500)
    return 1/(1+np.exp(-x))
def softmax(x):
    expX = np.exp(x)
    return expX/np.sum(expX, axis = 1,keepdims=True)

this is the model for classification

In [3]:
def initializeparameters_class(n0,n1,n2):
    W1=np.random.random((n1,n0))
    B1 = np.zeros((n1, 1))
    W2=np.random.random((n2,n1))
    B2 = np.zeros((n2, 1))
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def forwardpropogation_class(X,parameters):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=(np.dot(W1,X.T)+B1).T
    A1=relu(Z1)
    Z2=(np.dot(W2,A1.T)+B2).T
    A2=sig(Z2)
    A2 = np.clip(A2, 1e-10, 1 - 1e-10)
    A2 = np.nan_to_num(A2, nan=1e-10)
    forward_cache={
        'Z1' :Z1,
        'A1' :A1,
        'Z2' :Z2,
        'A2' :A2

    }
    return forward_cache
def costfunction_class(Y,A2):
    m=Y.shape[0]
    cost = -(1/m)*np.sum(Y*np.log(A2)+(1-Y)*np.log(1-A2))
    return cost
def backwardpropagation_class(X,Y,parameters,forward_cache):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=forward_cache['Z1']
    A1=forward_cache['A1']
    Z2=forward_cache['Z2']
    A2=forward_cache['A2']
    m=Y.shape[0]
    dZ2=A2-Y
    dW2=(1/m)*np.dot(dZ2.T,A1)
    dB2=(1/m)*np.sum(dZ2,axis=0,keepdims=True).T
    dA1=np.dot(dZ2,W2)
    dZ1=dA1*derrelu(Z1)
    dW1=(1/m)*np.dot(dZ1.T,X)
    dB1=(1/m)*np.sum(dZ1,axis=0,keepdims=True).T
    gradients={
        'dW1' :dW1,
        'dB1' :dB1,
        'dW2' :dW2,
        'dB2' :dB2
    }
    return gradients
def updateparameters_class(parameters,gradients,learningrate):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    dW1=gradients['dW1']
    dW2=gradients['dW2']
    dB1=gradients['dB1']
    dB2=gradients['dB2']
    W1-=learningrate*dW1
    B1-=learningrate*dB1
    W2-=learningrate*dW2
    B2-=learningrate*dB2
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def model_class(X,Y,n1,learningrate,iterations):
    n0=X.shape[1]
    n2=Y.shape[1]
    costlist=[]
    parameters=initializeparameters_class(n0,n1,n2)
    for i in range(iterations):
        forward_cache=forwardpropogation_class(X,parameters)
        cost=costfunction_class(Y,forward_cache['A2'])
        gradients=backwardpropagation_class(X,Y,parameters,forward_cache)
        parameters=updateparameters_class(parameters,gradients,learningrate)
        if(i%(iterations/10)==0):
            costlist.append(cost)
    return parameters,costlist

this is the model for multi-class classification

In [4]:
def initializeparameters_mulclass(n0,n1,n2):
    W1 = np.random.randn(n1, n0)*0.01
    B1 = np.zeros((n1, 1))
    
    W2 = np.random.randn(n2, n1)*0.01
    B2 = np.zeros((n2, 1))
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def forwardpropogation_mulclass(X,parameters):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=(np.dot(W1,X.T)+B1).T
    A1=tanh(Z1)
    Z2=(np.dot(W2,A1.T)+B2).T
    A2=softmax(Z2)
    forward_cache={
        'Z1' :Z1,
        'A1' :A1,
        'Z2' :Z2,
        'A2' :A2

    }
    return forward_cache
def costfunction_mulclass(Y,A2):
    m=Y.shape[0]
    cost = -(1/m)*np.sum(Y*np.log(A2))
    return cost
def backwardpropagation_mulclass(X,Y,parameters,forward_cache):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=forward_cache['Z1']
    A1=forward_cache['A1']
    Z2=forward_cache['Z2']
    A2=forward_cache['A2']
    m=Y.shape[0]
    dZ2=A2-Y
    dW2=(1/m)*np.dot(dZ2.T,A1)
    dB2=(1/m)*np.sum(dZ2,axis=0,keepdims=True).T
    dA1=np.dot(dZ2,W2)
    dZ1=dA1*dertanh(Z1)
    dW1=(1/m)*np.dot(dZ1.T,X)
    dB1=(1/m)*np.sum(dZ1,axis=0,keepdims=True).T
    gradients={
        'dW1' :dW1,
        'dB1' :dB1,
        'dW2' :dW2,
        'dB2' :dB2
    }
    return gradients
def updateparameters_mulclass(parameters,gradients,learningrate):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    dW1=gradients['dW1']
    dW2=gradients['dW2']
    dB1=gradients['dB1']
    dB2=gradients['dB2']
    W1-=learningrate*dW1
    B1-=learningrate*dB1
    W2-=learningrate*dW2
    B2-=learningrate*dB2
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def model_mulclass(X,Y,n1,learningrate,iterations):
    n0=X.shape[1]
    n2=Y.shape[1]
    costlist=[]
    parameters=initializeparameters_mulclass(n0,n1,n2)
    for i in range(iterations):
        forward_cache=forwardpropogation_mulclass(X,parameters)
        cost=costfunction_mulclass(Y,forward_cache['A2'])
        gradients=backwardpropagation_mulclass(X,Y,parameters,forward_cache)
        parameters=updateparameters_mulclass(parameters,gradients,learningrate)
        if(i%(iterations/10)==0):
            costlist.append(cost)
    return parameters,costlist

this is the model for regression

In [5]:
def initializeparameters_reg(n0,n1,n2):
    W1 = np.random.randn(n1, n0)*0.01
    B1 = np.zeros((n1, 1))
    
    W2 = np.random.randn(n2, n1)*0.01
    B2 = np.zeros((n2, 1))
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def forwardpropogation_reg(X,parameters):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=(np.dot(W1,X.T)+B1).T
    A1=tanh(Z1)
    Z2=(np.dot(W2,A1.T)+B2).T
    A2=Z2
    forward_cache={
        'Z1' :Z1,
        'A1' :A1,
        'Z2' :Z2,
        'A2' :A2

    }
    return forward_cache
def costfunction_reg(Y,A2):
    m=Y.shape[0]
    cost = (1/(2*m))*np.dot((A2-Y).T,(A2-Y))  
    return cost
def backwardpropagation_reg(X,Y,parameters,forward_cache):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    Z1=forward_cache['Z1']
    A1=forward_cache['A1']
    Z2=forward_cache['Z2']
    A2=forward_cache['A2']
    m=Y.shape[0]
    dZ2=A2-Y
    dW2=(1/m)*np.dot(dZ2.T,A1)
    dB2=(1/m)*np.sum(dZ2,axis=0,keepdims=True).T
    dA1=np.dot(dZ2,W2)
    dZ1=dA1*dertanh(Z1)
    dW1=(1/m)*np.dot(dZ1.T,X)
    dB1=(1/m)*np.sum(dZ1,axis=0,keepdims=True).T
    gradients={
        'dW1' :dW1,
        'dB1' :dB1,
        'dW2' :dW2,
        'dB2' :dB2
    }
    return gradients
def updateparameters_reg(parameters,gradients,learningrate):
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']
    dW1=gradients['dW1']
    dW2=gradients['dW2']
    dB1=gradients['dB1']
    dB2=gradients['dB2']
    W1-=learningrate*dW1
    B1-=learningrate*dB1
    W2-=learningrate*dW2
    B2-=learningrate*dB2
    parameters={
        'W1' :W1,
        'B1' :B1,
        'W2' :W2,
        'B2' :B2
    }
    return parameters
def model_reg(X,Y,n1,learningrate,iterations):
    n0=X.shape[1]
    n2=Y.shape[1]
    costlist=[]
    parameters=initializeparameters_reg(n0,n1,n2)
    for i in range(iterations):
        forward_cache=forwardpropogation_reg(X,parameters)
        cost=costfunction_reg(Y,forward_cache['A2'])
        gradients=backwardpropagation_reg(X,Y,parameters,forward_cache)
        parameters=updateparameters_reg(parameters,gradients,learningrate)
        if(i%(iterations/10)==0):
            costlist.append(cost)
    return parameters,costlist

In [ ]:
import socket
import threading
import numpy as np
import struct
host=socket.gethostbyname(socket.gethostname())
port=5060
server=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server.bind((host,port))
server.listen(4)
print('server is listening')
def handle_client(communication_socket,address):
    print('new connection ',address,' connected')
    shape_length_headerX = communication_socket.recv(10).decode('utf-8').strip()
    shape_lengthX = int(shape_length_headerX)
    shape_arrayX=communication_socket.recv(shape_lengthX).decode('utf-8')
    shapeX = tuple(map(int, shape_arrayX.split(',')))
    array_bitsX=communication_socket.recv(np.prod(shapeX) * 8)
    X=np.frombuffer(array_bitsX,dtype=np.float64)
    X=X.reshape(shapeX)
    shape_length_headerY = communication_socket.recv(10).decode('utf-8').strip()
    shape_lengthY = int(shape_length_headerY)
    shape_arrayY=communication_socket.recv(shape_lengthY).decode('utf-8')
    shapeY = tuple(map(int, shape_arrayY.split(',')))
    array_bitsY=communication_socket.recv(np.prod(shapeY) * 8)
    Y=np.frombuffer(array_bitsY,dtype=np.float64)
    Y=Y.reshape(shapeY)
    #start of machine learning
    choice=communication_socket.recv(2040).decode('utf-8')
    neurons = struct.unpack('i', communication_socket.recv(4))[0]
    learning_rate = struct.unpack('d', communication_socket.recv(8))[0]
    iter = struct.unpack('i', communication_socket.recv(4))[0]
    print('choice is ',choice)
    if choice=='1':
        parameters,costlist=model_reg(X,Y,neurons,learning_rate,iter)
    elif choice=='2':
        parameters,costlist=model_class(X,Y,neurons,learning_rate,iter)
    elif choice=='3':
        parameters,costlist=model_mulclass(X,Y,neurons,learning_rate,iter)
    W1=parameters['W1']
    W2=parameters['W2']
    B1=parameters['B1']
    B2=parameters['B2']

    array_shape = ','.join(map(str, W1.shape))
    shape_length = str(len(array_shape)).ljust(10)
    communication_socket.send(shape_length.encode('utf-8')) 
    communication_socket.send(array_shape.encode('utf-8'))
    W1_bytes = W1.tobytes()
    communication_socket.send(W1_bytes)
    print('W1 is',W1)
    
    array_shape2 = ','.join(map(str, W2.shape))
    shape_length2 = str(len(array_shape2)).ljust(10)
    communication_socket.send(shape_length2.encode('utf-8'))  
    communication_socket.send(array_shape2.encode('utf-8'))   


    W2_float64 = W2.astype(np.float64)  
    W2_bytes = W2_float64.tobytes()  

    print(f'Sending W2 with shape: {W2.shape} and {len(W2_bytes)} bytes')
    communication_socket.send(W2_bytes)  
    print('W2 is',W2)
    
    array_shape3 = ','.join(map(str, B1.shape))
    shape_length3 = str(len(array_shape3)).ljust(10)
    communication_socket.send(shape_length3.encode('utf-8'))  
    communication_socket.send(array_shape3.encode('utf-8'))  


    B1_float64 = B1.astype(np.float64)  
    B1_bytes = B1_float64.tobytes()  

    print(f'Sending B1 with shape: {B1.shape} and {len(B1_bytes)} bytes')
    communication_socket.send(B1_bytes)  
    print('B1 is',B1)
    
    array_shape4 = ','.join(map(str, B2.shape))
    shape_length4 = str(len(array_shape4)).ljust(10)
    communication_socket.send(shape_length4.encode('utf-8'))  
    communication_socket.send(array_shape4.encode('utf-8'))  


    B2_float64 = B2.astype(np.float64)  
    B2_bytes = B2_float64.tobytes()  

    print(f'Sending B2 with shape: {B2.shape} and {len(B2_bytes)} bytes')
    communication_socket.send(B2_bytes) 
    print('B2 is',B2)
    
    communication_socket.close()
    print('the neurons are:',neurons)
    print('the learning are:',learning_rate) 
    print('the number of iteration are:',iter)

    
while True:
    communication_socket,address=server.accept()
    thread=threading.Thread(target=handle_client,args=(communication_socket,address))
    thread.start()
    print('active connections ',threading.active_count()-1)


server is listening
new connection  ('127.0.0.1', 44998)  connected
active connections  7
choice is  1
W1 is [[-0.00258089 -0.00540269 -0.00864786]
 [-0.00388136 -0.01722356 -0.01056198]
 [-0.00435319 -0.00434062 -0.0042953 ]
 ...
 [-0.02955187 -0.01213315  0.00334843]
 [ 0.010333   -0.00940169  0.01207663]
 [-0.00939648 -0.00717928 -0.01298846]]
Sending W2 with shape: (1, 1000) and 8000 bytes
W2 is [[-6.18633326e-03 -1.34415355e-02 -3.28246468e-03 -4.72443257e-03
   3.79390142e-03 -1.19308017e-02 -2.43757215e-03  1.71436521e-02
  -1.58613005e-03 -8.96070121e-03  4.77014831e-03 -1.11689482e-02
   1.39843460e-03  6.11301535e-03 -1.20535529e-02  3.11868594e-03
   1.54510927e-02 -1.53263597e-03 -2.67558495e-03  5.84028409e-04
   6.68804765e-03 -1.66914744e-02  2.87854666e-02  5.73088882e-03
  -1.18609049e-03 -6.47227054e-03 -2.11442398e-02  5.95647090e-03
   7.15110870e-03 -1.39731813e-02  2.24617114e-03  1.42576751e-03
   2.26449214e-02 -7.16535163e-03  4.33863694e-03  1.70618230e-03
  -